In [9]:
import gym
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

In [10]:
env = gym.make('FrozenLake-v0')

In [11]:
input_size = env.observation_space.n
output_size = env.action_space.n
learning_rate = 0.1

In [12]:
X = tf.placeholder(shape=[1,input_size], dtype=tf.float32) #(1,16)
W = tf.Variable(tf.random_uniform([input_size, output_size], 0, 0.01)) #(16,4)
Qpred = tf.matmul(X,W) # (1,4)
Y = tf.placeholder(shape=[1, output_size], dtype=tf.float32)

loss = tf.reduce_sum(tf.square(Y-Qpred))
train = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)

In [13]:
dis = .99
num_episodes = 2000
rList = []

In [14]:
def one_hot(x):
    return np.identity(16)[x:x+1]
# one_hot(3)

In [17]:
init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    for i in range(num_episodes):
        s = env.reset()
        e = 1. / ((i/50)+10) # e-greedy 0.1 ==> ... 0 approximate
        rAll = 0
        done = False
        
        while not done:
            Qs = sess.run(Qpred, feed_dict={X:one_hot(s)})
            # exploit & exploration
            if np.random.rand(1) < e: # 10%확률로 탐험
                a = env.action_space.sample()
            else:
                a = np.argmax(Qs)

            s1, reward, done, _ = env.step(a)

            if done:
                Qs[0,a] = reward
            else:
                Qs1 = sess.run(Qpred, feed_dict={X:one_hot(s1)})
                Qs[0,a] = reward + dis * np.max(Qs1)
            sess.run(train, feed_dict={X:one_hot(s), Y:Qs})

            rAll += reward
            s = s1
        rList.append(rAll)
    
print(str(sum(rList)/num_episodes)+"%")
            

0.4765%
